In [ ]:
%load_ext autoreload
%autoreload 2

Todo's:

- Remove sentence limitation

In [ ]:
import pandas as pd

from datetime import datetime
import hashlib

from financial_report_analyzer.database_conntector import DatabaseConnector
from financial_report_analyzer.scraping import SECScraper
from financial_report_analyzer.model import ScoringModel
from financial_report_analyzer.content_extractor import TextExtractor

### Initialize

In [ ]:
sec_scraper = SECScraper()
model = ScoringModel()
connector = DatabaseConnector()

### Load Data

In [ ]:
connector.table_names

filings = connector.fetch_data("filings")
scores = connector.fetch_data("scores")

### Hashing

In [ ]:
def create_hash(texts: list):
    raw_text = "".join(texts)
    return hashlib.sha256(raw_text.encode()).hexdigest()

### Analyze Reports

In [ ]:
def merged(df, col1="ticker", col2="year"):
    return df[col1] + df[col2]


analyzed = (filings.pipe(merged)).isin((scores.pipe(merged)).tolist())

analyzed_filings = filings[analyzed]
not_analyzed_filings = filings[~analyzed]

In [ ]:
def create_report_scores(filing, sentences_limit=False):
    ticker = filing["ticker"]
    year = filing["year"]
    filing_url = filing["filing_url"]

    report = sec_scraper.fetch_report(filing_url)

    extractor = TextExtractor(report)
    sentences = extractor.get_sentences()
    text_hash = create_hash(sentences)

    if sentences_limit:
        sentences = sentences[:50]
    
    report_scores = model.calculate_report_scores(
        sentences[:50]
    )
    report_scores.update(
        {
            "ticker": ticker,
            "year": year,
            "analysis_timestamp": datetime.now(),
            "text_hash": text_hash,
        }
    )
    return report_scores

In [ ]:
new_report_scores = []

for _, filing in not_analyzed_filings.iterrows():
    report_scores = create_report_scores(filing, sentences_limit=True)
    new_report_scores.append(report_scores)

## Store scores in database

In [ ]:
upload_scores = pd.concat([scores, pd.DataFrame(new_report_scores)])

In [ ]:
connector.store_data(upload_scores)